In [1]:
from dotenv import load_dotenv, find_dotenv
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool
# import nest_asyncio

# nest_asyncio.apply()

_ = load_dotenv(find_dotenv())

In [2]:
all_inference_models = !curl -s https://huggingface.co/api/models | jq ".[].id"

In [2]:
from huggingface_hub import model_info


In [3]:

info = model_info("Qwen/Qwen2.5-VL-72B-Instruct", expand="inference")
info.inference


'warm'

In [52]:
vlm_model = InferenceClientModel(model_id="Qwen/Qwen2.5-VL-72B-Instruct",bill_to="VitalNest",
                                 timeout=300,temperature=.1, max_tokens=10000)

In [53]:
print(vlm_model([{"role":"user", "content":"what can you do ?"}]).content)

As a large language model, I can assist you in a variety of tasks, including but not limited to:

  1. Answering questions on a wide range of topics.
  2. Providing information and explanations on various subjects.
  3. Generating text, such as articles, stories, and poems.
  4. Translating text from one language to another.
  5. Offering suggestions and recommendations.
  6. Engaging in conversation and providing assistance with specific tasks.

Please let me know how I can assist you today.


In [11]:
from PIL import Image
from io import BytesIO
import base64
from pathlib import Path

In [10]:
def encode_image_to_base64(image_path):
    """
    Encodes an image file to a base64 string.

    Args:
        image_path (str): Path to the image file.

    Returns:
        str: Base64 encoded image string.
    """
    """Encodes a PIL image to a base64 string."""
    image = Image.open(image_path)
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


In [68]:
img_paths = [
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_42.png"
    ),
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_12.png"
    ),
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_1.png"
    ),
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_43.png"
    ),
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_44.png"
    ),
]

In [74]:
# grouped_images = [Image.open(fp=pth) for pth in img_paths]
grouped_images = [encode_image_to_base64(pth) for pth in img_paths]

In [75]:
text_query = """what critical illnesses are covered under optima restore?"""
chat_template = [
    {
        "role": "user",
        "content": [{"type": "image", "image": image} for image in grouped_images]
        + [{"type": "text", "text": text_query}],
    }
]


In [ ]:
# print(vlm_model(chat_template).content)

In [64]:
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="hyperbolic",
    bill_to="VitalNest",
)


In [ ]:
completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-VL-72B-Instruct",
    messages=chat_template,
)

print(completion.choices[0].message)

# completion = client.chat.completions.create(
#     model="Qwen/Qwen2.5-VL-72B-Instruct",
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {"type": "text", "text": "Describe this image in one sentence."},
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": "https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg"
#                     },
#                 },
#             ],
#         }
#     ],
# )

# print(completion.choices[0].message)


In [2]:
from smolagents import Tool

class SaveContent(Tool):
    name = "SaveContent"
    description = "Saves content to a file."
    inputs = {
        "content": {
            "type": "string",
            "description": "The content to save."
        },
        "filename": {
            "type": "string",
            "description": "The filename."
        }
    }
    output_type = "string"

    def forward(self, content: str, filename: str ):
        with open(filename, "w") as f:
            f.write(content)
        return f"Content saved to {filename}"



In [17]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), SaveContent()], 
                  model=InferenceClientModel(bill_to="VitalNest",max_tokens=150000),
                  additional_authorized_imports=["os", "requests", "bs4"],
                 max_steps=10, planning_interval=4,
                 )

In [ ]:
agent.run(f"""Use a headless browser agent to navigate to this page: https://www.hdfcergo.com/download/policy-wordings.
Retrieve all the policy wording documents under health category.""")

In [1]:
import torch
from PIL import Image
from transformers.utils.import_utils import is_flash_attn_2_available

from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor


In [ ]:

model = ColQwen2_5.from_pretrained(
    pretrained_model_name_or_path="vidore/colqwen2.5-v0.2",
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",  # or "mps" if on Apple Silicon
    attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
).eval()
processor = ColQwen2_5_Processor.from_pretrained("vidore/colqwen2.5-v0.2", use_fast=True)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:

# Your inputs
images = [
    Image.new("RGB", (128, 128), color="white"),
    Image.new("RGB", (128, 128), color="black"),
]
queries = [
    "Is attention really all you need?",
    "What is the amount of bananas farmed in Salvador?",
]

# Process the inputs
batch_images = processor.process_images(images).to(model.device)
batch_queries = processor.process_queries(queries).to(model.device)

# Forward pass
with torch.no_grad():
    image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)

scores = processor.score_multi_vector(query_embeddings, image_embeddings)
